In [20]:
import numpy as np
import pandas as pd

# Paths
RAW_PATH = "../data/hospital_length_of_stay_raw.csv"
CLEAN_PATH = "../data/hospital_length_of_stay_clean.csv"

# Columns
MISSING_FLAG_COLS = ["heart_rate", "systolic_bp", "lab_abnormalities"]

INT_LIKE_COLUMNS = [
    "age",
    "heart_rate",
    "systolic_bp",
    "lab_abnormalities",
    "num_comorbidities",
]

# Clinical plausibility bounds (clip)
CLIP_BOUNDS = {
    "heart_rate": (30, 220),
    "systolic_bp": (70, 250),
}


In [21]:
def clean_los_dataframe(
    df: pd.DataFrame,
    missing_flag_cols=MISSING_FLAG_COLS,
    clip_bounds=CLIP_BOUNDS,
    int_like_columns=INT_LIKE_COLUMNS,
) -> pd.DataFrame:
    """
    Clean LOS dataset:
      1) Clip extreme physiologic outliers
      2) Create missingness indicator flags
      3) Median-impute select numeric columns
      4) Round/cast selected columns to int for realism
    Returns a new cleaned DataFrame (does not mutate input).
    """
    df = df.copy()

    # 1) Clip outliers (NaNs pass through unchanged)
    for col, (low, high) in clip_bounds.items():
        if col in df.columns:
            df[col] = df[col].clip(lower=low, upper=high)

    # 2) Missing flags (must happen BEFORE imputation)
    for col in missing_flag_cols:
        if col in df.columns:
            df[f"{col}_missing"] = df[col].isna().astype(int)

    # 3) Median imputation
    for col in missing_flag_cols:
        if col in df.columns:
            df[col] = df[col].fillna(df[col].median())

    # 4) Integer realism (safe now that NaNs are handled)
    for col in int_like_columns:
        if col in df.columns:
            # Round first to avoid float-to-int surprises
            df[col] = df[col].round().astype(int)

    return df


In [ ]:
df_raw = pd.read_csv(RAW_PATH)
df_clean = clean_los_dataframe(df_raw)

# Quick validations (these should always pass)
assert df_clean[MISSING_FLAG_COLS].isna().sum().sum() == 0, "Imputation failed: still have NaNs in key cols."
for col in MISSING_FLAG_COLS:
    assert set(df_clean[f"{col}_missing"].unique()).issubset({0, 1}), f"Missing flag {col}_missing not binary."


heart_rate           0.1444
systolic_bp          0.1256
lab_abnormalities    0.1096
dtype: float64

heart_rate_missing           0.1444
systolic_bp_missing          0.1256
lab_abnormalities_missing    0.1096
dtype: float64

,age,num_comorbidities,heart_rate,systolic_bp,lab_abnormalities,length_of_stay_days,heart_rate_missing,systolic_bp_missing,lab_abnormalities_missing
count,2500.000000,2500.000000,2500.000000,2500.000000,2500.00000,2500.000000,2500.000000,2500.000000,2500.000000
mean,65.670400,2.245600,87.652000,131.326800,1.51960,11.951959,0.144400,0.125600,0.109600
std,15.922863,1.513604,20.663109,20.980946,1.21285,5.232723,0.351565,0.331464,0.312453
min,14.000000,0.000000,30.000000,70.000000,0.00000,3.350352,0.000000,0.000000,0.000000
25%,55.000000,1.000000,77.000000,119.000000,1.00000,9.166354,0.000000,0.000000,0.000000
50%,66.000000,2.000000,87.000000,132.000000,1.00000,11.037407,0.000000,0.000000,0.000000
75%,77.000000,3.000000,96.000000,144.000000,2.00000,13.444791,0.000000,0.000000,0.000000
max,126.000000,9.000000,220.000000,217.000000,7.00000,63.007113,1.000000,1.000000,1.000000


In [23]:
df_clean.to_csv(CLEAN_PATH, index=False)
print(f"Saved cleaned data to: {CLEAN_PATH}")

Saved cleaned data to: ../data/hospital_length_of_stay_clean.csv
